In [14]:
#Import + Configure Packages
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator', sizing_mode="stretch_width")

import hvplot.pandas
import holoviews as hv

hv.extension('bokeh')

In [15]:
#Choose colour blind friendly to aid readability + accessibility
pn.Row(
    pn.layout.HSpacer(height=50, styles={'background':"#ffbb69"}),
    pn.layout.HSpacer(height=50, styles={'background':"#69ffbb"}),
    pn.layout.HSpacer(height=50, styles={'background':"#bb69ff"}))

Row(sizing_mode='stretch_width')
    [0] HSpacer(height=50, styles={'background': '#ffbb69'})
    [1] HSpacer(height=50, styles={'background': '#69ffbb'})
    [2] HSpacer(height=50, styles={'background': '#bb69ff'})

In [16]:
#import owin global co2 emissions dataset
df = pd.read_csv("owid-co2-data.csv")

In [17]:
#Get overview of df to decide what dashboard should show
print(df.head()) # Chosen: Total CO2 per country, Total CO2 per continent, CO2 vs GDP of Country, CO2 Sources per Continent

       country  year iso_code  population  gdp  cement_co2  \
0  Afghanistan  1850      AFG   3752993.0  NaN         NaN   
1  Afghanistan  1851      AFG   3767956.0  NaN         NaN   
2  Afghanistan  1852      AFG   3783940.0  NaN         NaN   
3  Afghanistan  1853      AFG   3800954.0  NaN         NaN   
4  Afghanistan  1854      AFG   3818038.0  NaN         NaN   

   cement_co2_per_capita  co2  co2_growth_abs  co2_growth_prct  ...  \
0                    NaN  NaN             NaN              NaN  ...   
1                    NaN  NaN             NaN              NaN  ...   
2                    NaN  NaN             NaN              NaN  ...   
3                    NaN  NaN             NaN              NaN  ...   
4                    NaN  NaN             NaN              NaN  ...   

   share_global_other_co2  share_of_temperature_change_from_ghg  \
0                     NaN                                   NaN   
1                     NaN                                 0.157   

In [18]:
#Preprocessing to fill NA's with 0, rename country to continent and create GDP per capita column
df.isna().sum()
df = df.fillna(0)
df['gdp_per_capita'] = np.where(df['population']!= 0, df['gdp']/ df['population'], 0)
df.rename(columns = {'country': 'continent'}, inplace=True)

In [19]:
#make df interactive for dashboard to be interactive
idf = df.interactive()

In [20]:
# Create Visualisation for CO2 Emissions Over Time by Continent
# Define Panel widgets
year_slider = pn.widgets.IntSlider(name='Year Selector', start=1750, end=2020, step=1, value=2000)
year_slider #check it works

IntSlider(end=2020, name='Year Selector', sizing_mode='stretch_width', start=1750, value=2000)

In [21]:
# Radio buttons for CO2 measures
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name='Y axis', options=['co2', 'co2_per_capita'],button_type='success'
)
yaxis_co2 #check it works

RadioButtonGroup(button_type='success', name='Y axis', options=['co2', 'co2_per_capita'], sizing_mode='stretch_width', value='co2')

In [22]:
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_pipeline = (
    idf[
        (idf['year'] >= 1800) & (idf['year'] <= 2020) &
        (idf['continent'].isin(continents))
    ]
    .groupby(['continent', 'year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

co2_pipeline.head()

In [23]:
co2_plot = co2_pipeline.hvplot(x = 'year', by='continent', y=yaxis_co2, line_width=2, height=400, 
                                                                width=800, title="CO2 emission by continent")
co2_plot

In [24]:
#Table for Co2 emissions over time by continent
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, layout='fit_columns', pagination='remote', page_size = 13, width=780, height=435) 
co2_table

In [25]:
#CO2 v Population Scatterplot
#Remove non-countries
without_countries = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica', 'World', 'High-income countries', 
                     'OECD (GCP)', 'Non-OECD (GCP)', 'Asia (GCP)', 'Europe (GCP)', 'North America (GCP)', 'Upper-middle-income countries',
                    'Asia (excl. China and India)', 'European Union (28)', 'European Union (27) (GCP)', 'European Union (27)',
                    'Lower-middle-income countries', 'Europe (excl. EU-27)', 'Europe (excl. EU-28)', 'Middle East', 'North America (excl. USA)',
                    'Africa (GCP)', 'Middle East (GCP)', 'South America (GCP)', 'International transport', 'Oceania', 'Oceania (GCP)', 
                     'Central America (GCP)', 'Low-income countries']


co2_vs_population_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
        (~ (idf['continent'].isin(without_countries)))
    ]
    .groupby(['continent', 'year', 'population'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

co2_vs_population_scatterplot = co2_vs_population_scatterplot_pipeline.hvplot(x='population', 
                                                                y='co2', 
                                                                by='continent', 
                                                                size=80, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=True, 
                                                                height=450, 
                                                                width=800,
                                                                title='CO2 vs Population')
co2_vs_population_scatterplot

In [26]:
#I know it's lazy, but let's do the same, but for Country vs GDP.
#Remove non-countries
#Create pipeline
co2_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
        (~ (idf['continent'].isin(without_countries)))
    ]
    .groupby(['continent', 'year', 'gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x='gdp_per_capita', 
                                                                y='co2', 
                                                                by='continent', 
                                                                size=80, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=True, 
                                                                height=450, 
                                                                width=800,
                                                                title='CO2 vs GDP')
co2_vs_gdp_scatterplot

In [ ]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['coal_co2', 'oil_co2', 'gas_co2'], 
    button_type='success'
)

continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf['continent'].isin(continents_excl_world))
    ]
    .groupby(['year', 'continent'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='continent', 
                                                     y=yaxis_co2_source, 
                                                     title='CO2 source by continent')
co2_source_bar_plot

In [ ]:
#Layout using predefined Template
template = pn.template.FastListTemplate(
    title='Interactive CO2 emissions dashboard', 
    sidebar=[pn.pane.Markdown("# CO2 Emissions and Climate Change"), 
             pn.pane.Markdown("#### Ruh Roh. Oli made an interactive Co2 emmissions dashboard"), 
             pn.pane.PNG('dashb.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             year_slider],
    main=[pn.Row(pn.Column(yaxis_co2, 
                           co2_plot.panel(width=500), margin=(0,5)), 
                 co2_vs_population_scatterplot.panel(width=600), margin=(0,25)), 
          pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)

template.show()
template.servable()